In [1]:
import sys
from pathlib import Path
import torch
import requests
import json
import os 

model_name="large-v2"
pattern = "phoneme"



# In[2]:


from dataclasses import dataclass, field, replace
from typing import TYPE_CHECKING, Dict, Iterable, List, Optional, Sequence, Tuple, Union
import whisper_openAI.whisper as whisper
import torch
from whisper_openAI.whisper.tokenizer import Tokenizer, get_tokenizer
import torch
import torch.nn.functional as F
from torch import Tensor
import json

# We get the acoustic embeddings from Whisper Large V2
model,processor = whisper.load_model("large-v2")
# model,processor = whisper.load_model("medium")


In [2]:
speaker_id = 'M03'
test_speaker = speaker_id
pattern = "phoneme"

In [3]:
with open(f'Inference/gs_inferences/large-v2_hypo/torgo_train_{speaker_id}_{model_name}.json', "r") as file:  # Change the file path and name here
    train_data = json.load(file)

with open(f'Inference/gs_inferences/large-v2_hypo/torgo_val_{speaker_id}_{model_name}.json', "r") as valid_file:
    val_data = json.load(valid_file)

# Load the test set
with open(f'Inference/gs_inferences/large-v2_hypo/torgo_test_{speaker_id}_{model_name}.json', "r") as test_file:
    test_data = json.load(test_file)
    

from lit_llama.tokenizer import Tokenizer
from tqdm import tqdm


tokenizer_path: Path = Path("weights/tokenizer.model")
tokenizer = Tokenizer(tokenizer_path)
print(f"train has {len(train_data):,} samples")

train has 9,376 samples


In [4]:
    import torch
    old_folder = "tiny_hypo"
    old_data_train = torch.load(f'Inference/gs_inferences/{old_folder}/torgo_{test_speaker}_train.pt',map_location=torch.device('cpu'))
    old_data_val = torch.load(f'Inference/gs_inferences/{old_folder}/torgo_{test_speaker}_val.pt',map_location=torch.device('cpu'))
    old_data_test = torch.load(f'Inference/gs_inferences/{old_folder}/torgo_{test_speaker}_test.pt',map_location=torch.device('cpu'))
    
    
    # In[8]:
    
    
    # Print the lengths of the loaded data
    print(f'Length of old_data_train: {len(old_data_train)}')
    print(f'Length of old_data_val: {len(old_data_val)}')
    print(f'Length of old_data_test: {len(old_data_test)}')

Length of old_data_train: 9376
Length of old_data_val: 460
Length of old_data_test: 442


In [5]:
train_data[0]

{'F01-Session1-arrayMic-0009': {'temp': 0.74,
  'path': '/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0009.wav',
  'ground_truth': 'pat',
  'inference': ['',
   'ah',
   'ah!',
   'alright',
   'bartender 3',
   'berk',
   'bleep',
   'blurp',
   'burp',
   'burp!',
   'burps',
   'but',
   'bye!',
   'ehh',
   'ha',
   'hah',
   'hehe',
   'hot!',
   'hrrrgh',
   'huh',
   'hurt!',
   'huuuuh!',
   'im gonna go',
   'oh',
   'purr!',
   'rip camera man',
   'thanks for watching see you next time!',
   'ugh',
   'uh',
   'uhoh',
   'urgh!'],
  'source': 'NP-Torgo',
  'category': 'NP-Torgo',
  'time': 30.0}}

In [76]:
def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)
    
def process_train_data(train_data, old_data):
    # instruction = 'You are an ASR transcript selector. You have a few transcripts generated by an automatic speech recognition model. Your task is to generate the most likely transcript from them. If the generated transcripts have grammatical or logical errors, you will modify them accordingly to produce the most accurate and coherent transcript.'
    instruction = 'You are an ASR transcript selector. The first line is the phoneme recognition result. The rest of the lines are a few transcripts generated by an automatic speech recognition model. Your task is to generate the most likely transcript from them. If the generated transcripts have grammatical or logical errors, you will modify them accordingly to produce the most accurate and coherent transcript.'
    result = []

    for i in tqdm(range(len(train_data))):        
        for name in train_data[i].keys():
            ip = train_data[i][name]
        # print(ip)
        inference = ip['inference']
        gt = ip['ground_truth']
        path = ip['path']

        # Find the corresponding phoneme in the dataframe
        matching_entry = df[df['audio'] == path]

        phoneme = matching_entry["phoneme"].values[0]
        
        # Removing the ground_truth, if present among the inferences for the prompt
        if gt in inference:
            inference.remove(gt)
                
        # Joining the inputs with '\n'
        for_input = f"Phoneme Results: {phoneme} \n" + "Possible transcripts: \n" + '\n'.join(inference[:15])  # Add phoneme to the beginning of the input

        print(path)
        print(f"{for_input} \n")
        # The prompt follows the Alpaca template
        full_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{for_input}\n\n### Response:"""
        full_prompt_and_response = full_prompt + gt

        encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=2048, eos=False)
        encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=2048)
        labels = encoded_full_prompt_and_response.clone()
        labels_with_masked_input = encoded_full_prompt_and_response.clone()
        labels_with_masked_input[:len(encoded_full_prompt)] = -1
        
        audio_features = old_data[i]["audio_features"]
        
        result.append({**ip, 'index': name, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels, 'labels_with_masked_input': labels_with_masked_input, 'audio_features': audio_features.bfloat16()})
    
    return result

In [74]:
import pandas as pd
df = pd.read_csv('data_preparation/torgo_phoneme.csv')
df

,session,audio,text,speaker_id,test_data,phoneme
0,F01-Session1-arrayMic-0006,/work/van-speech-nlp/data/torgo/F01/Session1/w...,STICK,F01,1,d e r t
1,F01-Session1-arrayMic-0008,/work/van-speech-nlp/data/torgo/F01/Session1/w...,EXCEPT IN THE WINTER WHEN THE OOZE OR SNOW OR ...,F01,1,a s ɛ ɪ n n ə i l i uː r ɑ s uː ɑ a h i v ɛ n k
2,F01-Session1-arrayMic-0009,/work/van-speech-nlp/data/torgo/F01/Session1/w...,PAT,F01,0,p aː
3,F01-Session1-arrayMic-0010,/work/van-speech-nlp/data/torgo/F01/Session1/w...,UP,F01,1,aː b
4,F01-Session1-arrayMic-0011,/work/van-speech-nlp/data/torgo/F01/Session1/w...,MEAT,F01,0,m eɪ
...,...,...,...,...,...,...
16389,MC04-Session2-arrayMic-1022,/work/van-speech-nlp/data/torgo/MC04/Session2/...,WART,MC04,0,NaN
16390,MC04-Session2-arrayMic-1023,/work/van-speech-nlp/data/torgo/MC04/Session2/...,THE FAMILY REQUESTS THAT FLOWERS BE OMITTED,MC04,0,NaN
16391,MC04-Session2-arrayMic-1024,/work/van-speech-nlp/data/torgo/MC04/Session2/...,SHIP,MC04,0,NaN
16392,MC04-Session2-arrayMic-1026,/work/van-speech-nlp/data/torgo/MC04/Session2/...,WHY,MC04,0,NaN


In [77]:
split = "train"
result = process_train_data(, old_data_train)
torch.save(result,f'Inference/gs_inferences/torgo_{speaker_id}_{model_name}_{pattern}_{split}.pt')
print(f"Processed {split} data and saved checkpoint for {speaker_id}")

100%|██████████| 4/4 [00:00<00:00, 159.11it/s]

/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0009.wav
Phoneme Results: p aː 
Possible transcripts: 

ah
ah!
alright
bartender 3
berk
bleep
blurp
burp
burp!
burps
but
bye!
ehh
ha 

/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0012.wav
Phoneme Results: i 
Possible transcripts: 
eat
great
heh heeh
hey
hey hey
lets not get ahead of ourselves
little asleep
neat
right
wait
wait wait
we
ねえ。 nee 

/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0015.wav
Phoneme Results: eː 
Possible transcripts: 
a
a!
and
aye
aye!
ayeee
ayyy
ayyyy!
ayyyyyy
bye
eeeh
eh
ehh
ehhh
ehhhhh 

/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0016.wav
Phoneme Results: f aː n 
Possible transcripts: 
food
food!
i
sword
the world
wha
what
what!
wood
woop!
wut 

Processed train data and saved checkpoint for M03
